In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from langchain.vectorstores import Chroma
from chromadb.config import Settings
from langchain.embeddings import HuggingFaceInstructEmbeddings


INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [3]:
embeddings=HuggingFaceInstructEmbeddings(
    query_instruction='Represent the legal question for retrieving evidence documents:',
    model_name='C:/Users/ngoph/Desktop/luanvan/model'
)

c:\Users\ngoph\.conda\envs\me\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: C:/Users/ngoph/Desktop/luanvan/model
Load pretrained SentenceTransformer: C:/Users/ngoph/Desktop/luanvan/model
load INSTRUCTOR_Transformer
max_seq_length  512
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


In [4]:
chroma = Chroma(
    collection_name='law_documents',
    client_settings=Settings(chroma_api_impl="rest",
                                        chroma_server_host="localhost",
                                        chroma_server_http_port="8000",
                                        chroma_server_ssl_enabled=False
                                        ),
    embedding_function=embeddings
)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [5]:
chroma.search('Rút bảo hiểm xã hội',search_type='mmr')

[Document(page_content='Điều 109. Hồ sơ hưởng bảo hiểm xã hội một lần\n1. Sổ bảo hiểm xã hội.\n2. Đơn đề nghị hưởng bảo hiểm xã hội một lần của người lao động.\n3. Đối với người ra nước ngoài để định cư phải nộp thêm bản sao giấy xác nhận của cơ quan có thẩm quyền về việc thôi quốc tịch Việt Nam hoặc bản dịch tiếng Việt được chứng thực hoặc công chứng một trong các giấy tờ sau đây:\na) Hộ chiếu do nước ngoài cấp;\nb) Thị thực của cơ quan nước ngoài có thẩm quyền cấp có xác nhận việc cho phép nhập cảnh với lý do định cư ở nước ngoài;\nc) Giấy tờ xác nhận về việc đang làm thủ tục nhập quốc tịch nước ngoài; giấy tờ xác nhận hoặc thẻ thường trú, cư trú có thời hạn từ 05 năm trở lên của cơ quan nước ngoài có thẩm quyền cấp.\n4. Trích sao hồ sơ bệnh án trong trường hợp quy định tại điểm c khoản 1 Điều 60 và điểm c khoản 1 Điều 77 của Luật này.\n5. Đối với người lao động quy định tại Điều 65 và khoản 5 Điều 77 của Luật này thì hồ sơ hưởng trợ cấp một lần được thực hiện theo quy định tại khoản

Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.



Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [14]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [15]:
prompt_template = """Sử dụng các đoạn văn bản sau để trả lời câu hỏi ở cuối. Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố gắng tạo ra một câu trả lời.

{context}

Câu hỏi: {question}
Trả lời bằng tiếng Việt:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [16]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma.as_retriever(),chain_type_kwargs=chain_type_kwargs)

In [17]:
query = "Điều kiện để đăng ký bảo hiểm xã hội?"

In [19]:
qa({'query':query})

Token indices sequence length is longer than the specified maximum sequence length for this model (3876 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\ngoph\.conda\envs\me\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 3876, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


IndexError: index out of range in self